In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,1353.18,-0.01,0.00,-0.00,-0.00,-0.00,-0.00,0.00
3,-0.01,327.49,-0.00,0.00,0.00,0.00,0.00,-0.00
4,0.00,-0.00,68.15,-0.00,-0.00,0.00,-0.00,0.00
5,-0.00,0.00,-0.00,13.56,0.00,0.00,0.00,-0.00
6,-0.00,0.00,-0.00,0.00,2.64,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,0.54,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.16,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00003,-0.00004,-0.00002,-0.00005,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00004,0.00000,-0.00003
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00003
5,-0.00003,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00002,0.00004,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00005,0.00000,-0.00002,0.00001,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.010340276223341731

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.29188891e+05 1.31330146e+04 5.53081334e+02 2.10290580e+01
 7.46130160e-01 2.79894786e-02 1.84655994e-03 1.82646562e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999302,-0.036638,-0.007133,-0.001418,-0.000277,-0.000056,-0.000017,-0.000005
1,0.036344,0.998649,-0.036464,-0.006896,-0.001338,-0.000272,-0.000079,-0.000025
2,0.008380,0.035853,0.998423,-0.041634,-0.007722,-0.001564,-0.000455,-0.000138
3,0.001996,0.008240,0.040886,0.997790,-0.050653,-0.009853,-0.002840,-0.000872
4,0.000483,0.001986,0.009567,0.049394,0.996011,-0.070818,-0.019406,-0.005908
5,0.000126,0.000516,0.002481,0.012509,0.066169,0.985067,-0.152893,-0.041531
6,0.000051,0.000206,0.000994,0.004992,0.025738,0.132038,0.944051,-0.301072
7,0.000034,0.000138,0.000663,0.003336,0.017148,0.084222,0.291557,0.952678


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006979180699342447,
 0.00135089641335151,
 0.0015769106444264436,
 0.0022100491628447294,
 0.003988984839322196,
 0.014932583208173034,
 0.05594850722882594,
 0.04732204331209677]